## **Carga de modelos e inferencia con Hugging Face**


### Configuración


Para este cuaderno, usaremos las siguientes librerías:

* [`torch`](https://pytorch.org/) para aprendizaje profundo y modelado de redes neuronales.
* [`transformers`](https://huggingface.co/transformers/) para acceder a modelos preentrenados y realizar diversas tareas de NLP con facilidad.


### Instalación de las librerías requeridas


In [ ]:
#!pip install torch
#!pip install transformers

### Importación de las librerías necesarias

*Se recomienda importar todas las librerías requeridas en un solo lugar (aquí):*


In [ ]:
from transformers import pipeline
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Puedes usar esta sección para suprimir advertencias generadas por tu código:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

### **Clasificación de texto con DistilBERT**


#### **Cargar el modelo y el tokenizador**

Primero, inicializamos un tokenizador y un modelo para análisis de sentimiento usando DistilBERT afinado en el conjunto de datos SST-2. 
Esta configuración es útil para tareas donde necesitas clasificar rápidamente el sentimiento de un fragmento de texto con un modelo transformer eficiente y preentrenado.


In [ ]:
# Carga del tokenizador y del modelo
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
modelo = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

#### Preprocesamiento del texto de entrada

Tokeniza el texto de entrada y conviértelo a un formato adecuado para el modelo:

In [ ]:
# Texto simple
text = "Congratulations! You've won a free ticket to the Bahamas. Reply WIN to claim."

# Tokenización del texto de entrada
inputs = tokenizer(text, return_tensors="pt")

print(inputs)

Los IDs de los tokens son los índices de los tokens en el vocabulario. El campo `attention_mask` es esencial para procesar correctamente las secuencias con padding, asegurando un cálculo eficiente y manteniendo el rendimiento del modelo. 

Incluso cuando no hay tokens enmascarados, ayuda al modelo a diferenciar entre contenido real y relleno, lo cual es crítico para un procesamiento preciso y eficiente.


### Realizar la inferencia

El contexto `torch.no_grad()` se utiliza para desactivar el cálculo de gradientes. Esto reduce el consumo de memoria y acelera el cálculo, ya que no se necesitan gradientes durante la inferencia (es decir, cuando no estás entrenando el modelo). 

La sintaxis `**inputs` se usa para desempaquetar un diccionario de argumentos de palabra clave en Python, en el contexto de `model(**inputs)`:


In [ ]:
# Realiza la inferencia
with torch.no_grad():
    outputs = modelo(**inputs)

Otra forma es pasar `input_ids` y `attention_mask` como parámetros separados:


In [ ]:
#modelo(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])

#### Obtener los logits

Los logits son las predicciones crudas y no normalizadas del modelo. Extraigámoslos de la salida del modelo para procesarlos posteriormente, como determinar la clase predicha o calcular probabilidades.


In [ ]:
logits = outputs.logits
logits.shape

#### Postprocesamiento de la salida

Convierte los logits en probabilidades y obtén la clase predicha:


In [ ]:
# Convierte logits en probabilidades
probs = torch.softmax(logits, dim=-1)

# Obtiene la clase predicha
predicted_class = torch.argmax(probs, dim=-1)

# Mapea la clase predicha a la etiqueta
labels = ["NEGATIVE", "POSITIVE"]
predicted_label = labels[predicted_class]

print(f"Etiqueta predicha: {predicted_label}")

### **Generación de texto con GPT-2** 


#### Cargar el tokenizador

Carga el tokenizador preentrenado de GPT-2. El tokenizador convierte el texto en tokens que el modelo puede entender.


In [ ]:
# Carga del tokenizador y del modelo
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

Carga el modelo GPT-2 con una cabecera de modelado de lenguaje. El modelo genera texto basándose en los tokens de entrada.


In [ ]:
# Carga el tokenizador y el modelo

modelo = GPT2LMHeadModel.from_pretrained("gpt2")

#### Preprocesamiento del texto de entrada

Tokeniza el texto de entrada y conviértelo a un formato adecuado para el modelo, obteniendo los IDs de los tokens:

In [ ]:
# Prompt
prompt = "Once upon a time"

# Tokeniza el texto de entrada
inputs = tokenizer(prompt, return_tensors="pt")
inputs

### **Realiza la inferencia**

Genera texto usando el modelo

`inputs:` IDs de token de entrada provenientes del tokenizador

`attention_mask:` Máscara que indica a qué tokens debe prestar atención

`pad_token_id:` ID del token de relleno, establecido como el ID de fin de secuencia

`max_length:` Longitud máxima de las secuencias generadas

`num_return_sequence:` Número de secuencias a generar



In [ ]:
# Genera texto
output_ids = modelo.generate(
    inputs.input_ids, 
    attention_mask=inputs.attention_mask,
    pad_token_id=tokenizer.eos_token_id,
    max_length=50, 
    num_return_sequences=1
)

output_ids

o


```python
with torch.no_grad():
    outputs = modelo(**inputs) 

outputs


#### Postprocesamiento de la salida

Decodifica los tokens generados para obtener el texto:

In [ ]:
# Decodifica el texto generado
generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print(generated_text)

### **Función `pipeline()` de Hugging Face**

La función `pipeline()` de la librería `transformers` de Hugging Face es una API de alto nivel diseñada para simplificar el uso de modelos preentrenados en distintas tareas de procesamiento de lenguaje natural (NLP). 

Abstrae las complejidades de la carga de modelos, la tokenización, la inferencia y el postprocesamiento, permitiendo a los usuarios realizar tareas complejas de NLP con sólo unas pocas líneas de código.

#### Definición

```python
transformers.pipeline(
    task: str,
    model: Optional = None,
    config: Optional = None,
    tokenizer: Optional = None,
    feature_extractor: Optional = None,
    framework: Optional = None,
    revision: str = 'main',
    use_fast: bool = True,
    model_kwargs: Dict[str, Any] = None,
    **kwargs
)
```

#### Parámetros

* **task**: `str`

  * La tarea a realizar, como `"text-classification"`, `"text-generation"`, `"question-answering"`, etc.
  * Ejemplo: `"text-classification"`

* **model**: `Optional`

  * El modelo a usar. Puede ser un identificador de modelo en el hub de Hugging Face (cadena), la ruta a un directorio con archivos de modelo, o una instancia de modelo ya cargada.
  * Ejemplo: `"distilbert-base-uncased-finetuned-sst-2-english"`

* **config**: `Optional`

  * La configuración a utilizar. Puede ser una cadena, la ruta a un directorio, o un objeto de configuración ya cargado.
  * Ejemplo: `{"output_attentions": True}`

* **tokenizer**: `Optional`

  * El tokenizador a usar. Puede ser una cadena, la ruta a un directorio, o una instancia de tokenizador ya cargada.
  * Ejemplo: `"bert-base-uncased"`

* **feature\_extractor**: `Optional`

  * El extractor de características para tareas que lo requieran (por ejemplo, procesamiento de imágenes).
  * Ejemplo: `"facebook/detectron2"`

* **framework**: `Optional`

  * El framework a usar, `"pt"` para PyTorch o `"tf"` para TensorFlow. Si no se especifica, se infiere automáticamente.
  * Ejemplo: `"pt"`

* **revision**: `str`, valor por defecto `'main'`

  * La versión específica del modelo a usar (rama, etiqueta o hash de commit).
  * Ejemplo: `"v1.0"`

* **use\_fast**: `bool`, valor por defecto `True`

  * Indica si se debe usar la versión rápida del tokenizador si está disponible.
  * Ejemplo: `True`

* **model\_kwargs**: `Dict[str, Any]`, valor por defecto `None`

  * Argumentos adicionales de palabra clave que se pasan al modelo durante la inicialización.
  * Ejemplo: `{"output_hidden_states": True}`

* **kwargs**: `Any`

  * Argumentos adicionales que se pasan a los componentes del pipeline.

#### Tipos de tareas

La función `pipeline()` admite una amplia gama de tareas de NLP. A continuación, algunas de las más comunes:

1. **Clasificación de texto**: `text-classification`

   * **Propósito**: Clasificar texto en categorías predefinidas.
   * **Casos de uso**: Análisis de sentimiento, detección de spam, clasificación de temas.

2. **Generación de texto**: `text-generation`

   * **Propósito**: Generar texto coherente a partir de un prompt dado.
   * **Casos de uso**: Escritura creativa, generación de diálogos, completado de historias.

3. **Pregunta-Respuesta**: `question-answering`

   * **Propósito**: Responder preguntas basadas en un contexto dado.
   * **Casos de uso**: Sistemas de Q\&A, recuperación de información en documentos.

4. **Reconocimiento de Entidades Nombradas (NER)**: `ner` (o `token-classification`)

   * **Propósito**: Identificar y clasificar entidades nombradas (personas, organizaciones, ubicaciones) en el texto.
   * **Casos de uso**: Extracción de información estructurada de texto no estructurado.

5. **Resumen**: `summarization`

   * **Propósito**: Resumir textos largos en resúmenes más cortos y coherentes.
   * **Casos de uso**: Resumen de documentos, resumen de noticias.

6. **Traducción**: `translation_xx_to_yy` (por ejemplo, `translation_en_to_fr`)

   * **Propósito**: Traducir texto de un idioma a otro.
   * **Casos de uso**: Traducción de idiomas, aplicaciones multilingües.

7. **Rellenar máscara**: `fill-mask`

   * **Propósito**: Predecir palabras enmascaradas dentro de una oración (útil para modelado de lenguaje enmascarado).
   * **Casos de uso**: Tareas de modelado de lenguaje, análisis de predicciones del modelo.

8. **Clasificación sin entrenamiento previo (Zero-Shot)**: `zero-shot-classification`

   * **Propósito**: Clasificar texto en categorías sin datos de entrenamiento específicos para esas categorías.
   * **Casos de uso**: Tareas de clasificación flexibles y adaptables.

9. **Extracción de características**: `feature-extraction`

   * **Propósito**: Extraer representaciones (estados ocultos) del texto.
   * **Casos de uso**: Tareas posteriores que requieran representaciones de texto, como clustering, cálculo de similitud o entrenamiento de modelos personalizados.


### **Ejemplo 1: Clasificación de texto usando `pipeline()`**

En este ejemplo, usaremos la función `pipeline()` para realizar clasificación de texto. Cargaremos un modelo preentrenado de clasificación de texto y lo usaremos para clasificar un texto de ejemplo.

#### **Cargar el modelo de clasificación de texto**

Inicializamos el pipeline para la tarea `text-classification`, especificando el modelo `"distilbert-base-uncased-finetuned-sst-2-english"`. Este modelo está afinado para análisis de sentimiento.

#### **Clasificar el texto de ejemplo**

Usamos `classifier` para clasificar el texto de ejemplo:



In [ ]:
# Carga un modelo genérico de clasificación de texto
classifier = pipeline("text-classification", model="distilbert-base-uncased-finetuned-sst-2-english")

# Clasifica un texto de ejemplo
result = classifier("Congratulations! You've won a free ticket to the Bahamas. Reply WIN to claim.")
print(result)

#### Salida

La salida será una lista de diccionarios, donde cada diccionario contiene:

* `label`: La etiqueta predicha (por ejemplo, `"POSITIVE"` o `"NEGATIVE"`).
* `score`: La puntuación de confianza de la predicción.

### **Ejemplo 2: Detección de idioma usando `pipeline()`**

En este ejemplo, usaremos la función `pipeline()` para realizar detección de idioma. Cargaremos un modelo preentrenado de detección de idioma y lo usaremos para identificar el idioma de un texto de muestra.

#### **Cargar el modelo de detección de idioma**

Inicializamos el pipeline para la tarea `text-classification`, especificando el modelo `"papluca/xlm-roberta-base-language-detection"`. Este modelo está afinado para detección de idioma.

#### **Detectar el idioma del texto de ejemplo**

Usamos `classifier` para detectar el idioma del texto:


In [ ]:
from transformers import pipeline

classifier = pipeline("text-classification", model="papluca/xlm-roberta-base-language-detection")
result = classifier("Bonjour, comment ça va?")
print(result)

#### Salida

La salida será una lista de diccionarios, donde cada diccionario contiene:

* `label`: La etiqueta de idioma predicha (por ejemplo, `"fr"` para francés).
* `score`: La puntuación de confianza de la predicción.

### **Ejemplo 3: Generación de texto usando `pipeline()`**

En este ejemplo, usaremos la función `pipeline()` para realizar generación de texto. Cargaremos un modelo preentrenado de generación de texto y lo usaremos para generar texto a partir de un prompt dado.

#### **Inicializa el pipeline de generación de texto**

Inicializamos el pipeline para la tarea `text-generation`, especificando el modelo `"gpt2"`. GPT-2 es un modelo conocido para tareas de generación de texto.

In [ ]:
# Inicializa el pipeline de generación de texto con GPT-2
generator = pipeline("text-generation", model="gpt2")

#### **Generar texto a partir de un prompt dado**

Usamos el generador para producir texto a partir del prompt: "Once upon a time". Especificamos `max_length=50` y `truncation=True` para limitar el texto generado a 50 tokens, y `num_return_sequences=1` para generar una única secuencia. 

La función `generator` devuelve el texto generado, que luego se imprime.


In [ ]:
# Genera texto a partir de un prompt dado
prompt = "Once upon a time"
result = generator(prompt, max_length=50, num_return_sequences=1, truncation=True)

# Imprime el texto generado
print(result[0]['generated_text'])

#### Salida

La salida será una lista de diccionarios, donde cada diccionario contiene:

* `generated_text`: El texto generado a partir del prompt de entrada.


### **Ejemplo 4: Generación de texto con T5 usando `pipeline()`**

En este ejemplo, usaremos la función `pipeline()` para realizar generación de texto-a-texto con el modelo T5. Cargaremos un modelo T5 preentrenado y lo usarás para traducir una frase de inglés a francés a partir de un prompt dado.

#### Inicializa el pipeline de generación de texto-a-texto

Inicializamos el pipeline para la tarea `text2text-generation`, especificando el modelo `"t5-small"`. T5 es un modelo versátil que puede realizar varias tareas de generación de texto-a-texto, incluida la traducción.


In [ ]:
# Inicializa el pipeline de generación de texto-a-texto con T5
generator = pipeline("text2text-generation", model="t5-small")

#### Genera texto basado en un prompt dado:

Usamos el generador para traducir una oración del inglés al francés basándonos en el prompt: "translate English to French: How are you?". 

Especificamos `max_length=50` para limitar el texto generado a 50 tokens y `num_return_sequences=1` para generar una única secuencia. La función `generator` devuelve el texto traducido, que luego se imprime.


In [ ]:
# Genera el texto basado en un prompt dado
prompt = "translate English to French: How are you?"
result = generator(prompt, max_length=50, num_return_sequences=1)

# Imprime el texto traducido
print(result[0]['generated_text'])

#### Salida

La salida será una lista de diccionarios, donde cada diccionario contiene:

* `generated_text`: El texto generado (traducido) a partir del prompt de entrada.

#### Beneficios de usar `pipeline()`

* **Reducción de código repetitivo**: Simplifica el código necesario para realizar tareas de PLN.
* **Mejora de la legibilidad**: Hace que el código sea más claro y expresivo.
* **Eficiencia de tiempo**: Ahorra tiempo al encargarse automáticamente de la carga del modelo, tokenización, inferencia y postprocesamiento.
* **API consistente**: Ofrece una interfaz uniforme para distintas tareas, lo que facilita la experimentación y el prototipado rápido.
* **Manejo automático del framework**: Gestiona automáticamente el framework subyacente (TensorFlow o PyTorch).

#### Cuándo usar `pipeline()`

* **Prototipado rápido**: Cuando necesitas prototipar rápidamente una aplicación de NLP o experimentar con distintos modelos.
* **Tareas sencillas**: Para realizar tareas comunes de PLN que están bien soportadas por `pipeline()`.
* **Despliegue**: Al desplegar modelos de NLP en entornos donde la simplicidad y facilidad de uso son cruciales.

#### Cuándo evitar `pipeline()`

* **Tareas personalizadas**: Cuando requieres llevar a cabo tareas muy específicas que `pipeline()` no soporta bien.
* **Optimización de rendimiento**: Si necesitas control detallado sobre el modelo y el proceso de tokenización para optimizar el rendimiento o casos de uso particulares.

#### **Ejercicio: tarea de rellenar máscaras con BERT usando `pipeline()`**

En este ejercicio usarás la función `pipeline()` para realizar una tarea de fill-mask con el modelo BERT. Cargarás un modelo BERT preentrenado y lo emplearás para predecir la palabra enmascarada en una frase dada.

#### **Instrucciones**

1. **Inicializa** el pipeline de fill-mask con el modelo BERT.
2. **Crea** un prompt que incluya un token enmascarado.
3. **Genera** texto rellenando el token enmascarado.
4. **Imprime** el texto resultante con las predicciones.

In [ ]:
# Tus respuestas